## テスト


### 対話的な実行例をテストする - doctest

- [doctest --- 対話型の Python の例をテストする](https://docs.python.org/ja/3.13/library/doctest.html)


In [13]:
"""
与えられた引数について、a / bを行う関数です

>>> div(5, 2)
2.5
"""


def div(a: int, b: int) -> float:
    """
    答えは少数で返ってきます

    >>> [div(n, 2) for n in range(5)]
    [0.0, 0.5, 1.0, 1.5, 2.0]

    Args:
        a (int): 割られる数
        b (int): 割る数

    Returns:
        float: 商
    """
    return a / b


import doctest

# tst = doctest.testmod()
# repr(tst)

In [14]:
# エラーテスト


def div(a: int, b: int) -> float:
    """
    第二引数が0だった場合は、ゼロ除算エラーが発生します

    >>> div(1, 0)
    Traceback (most recent call last):
        ...
    ZeroDivisionError: division by zero

    Args:
        a (int): 割られる数
        b (int): 割る数

    Returns:
        float: 商
    """
    return a / b


doctest.testmod(verbose=True)

Trying:
    div(5, 2)
Expecting:
    2.5
ok
Trying:
    div(1, 0)
Expecting:
    Traceback (most recent call last):
        ...
    ZeroDivisionError: division by zero
ok
2 items passed all tests:
   1 test in __main__
   1 test in __main__.div
2 tests in 2 items.
2 passed.
Test passed.


TestResults(failed=0, attempted=2)

In [24]:
# Jupyterの場合はimportとかではないので、globsで対象のテスト関数を定義しておいたり、globals()で取得することができるようにしておく
# また、module_relativeをFalseに設定することで、OSに依存したパスを設定することができる
doctest.testfile("./tmpfiles/test.txt", module_relative=False, globs={"div": div})

**********************************************************************
File "./tmpfiles/test.txt", line 5, in test.txt
Failed example:
    div(6, 2)
Expected:
    4.0
Got:
    3.0
**********************************************************************
1 item had failures:
   1 of   1 in test.txt
***Test Failed*** 1 failure.


TestResults(failed=1, attempted=1)

### ユニットテストフレームワークを利用する - unittest

- テストの自動化
- 初期設定と終了処理の共有
- テストの分類
- テスト実行と結果レポートの分離


In [25]:
def add(a: int, b: int) -> int:
    """2つの整数の合計を取得する
    あえて失敗させるためのバグを仕込む
    """
    if a == 1 and b == 3:
        return 3
    elif a == 3 and b == 3:
        return 7
    return a + b

In [31]:
import unittest


class AddTest(unittest.TestCase):
    def test_get_the_sum_of_two_integers(self):
        """
        add関数のテストコード
        """
        actual = add(1, 2)
        expected = 3
        self.assertEqual(actual, expected)


unittest.main(argv=["first-arg-is-ignored"], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [32]:
class AddTest(unittest.TestCase):
    def test_get_the_sum_of_two_integers(self):
        """
        add関数のテストコード、失敗するケース
        """
        actual = add(1, 3)
        expected = 4
        self.assertEqual(actual, expected)


unittest.main(argv=["first-arg-is-ignored"], exit=False)

F
FAIL: test_get_the_sum_of_two_integers (__main__.AddTest.test_get_the_sum_of_two_integers)
add関数のテストコード、失敗するケース
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_21187/2097895976.py", line 8, in test_get_the_sum_of_two_integers
    self.assertEqual(actual, expected)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
AssertionError: 3 != 4

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


In [33]:
# 複数のアサーションを行いたい場合
class AddTest(unittest.TestCase):
    def test_get_the_sum_of_two_integers(self):
        examples = [
            [1, 2, 3],
            [1, 3, 4],
            [3, 3, 6],
        ]
        for idx, exsample in enumerate(examples):
            a, b, expected = exsample
            with self.subTest(f"{a} + {b} = {expected}", idx=idx):
                self.assertEqual(add(a, b), expected)


unittest.main(argv=["first-arg-is-ignored"], exit=False)

FF
FAIL: test_get_the_sum_of_two_integers (__main__.AddTest.test_get_the_sum_of_two_integers) [1 + 3 = 4] (idx=1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_21187/1848870928.py", line 12, in test_get_the_sum_of_two_integers
    self.assertEqual(add(a, b), expected)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
AssertionError: 3 != 4

FAIL: test_get_the_sum_of_two_integers (__main__.AddTest.test_get_the_sum_of_two_integers) [3 + 3 = 6] (idx=2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_21187/1848870928.py", line 12, in test_get_the_sum_of_two_integers
    self.assertEqual(add(a, b), expected)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
AssertionError: 7 != 6

----------------------------------------------------------------------
Ran 1 te

#### テストを実行する前後処理


In [37]:
class SetUpAndSetUpClassTest(unittest.TestCase):
    def setUp(self):
        # テスト毎に呼び出されるメソッド
        print("setUp実行")

    @classmethod
    def setUpClass(cls):
        # テストが実行される前に1度だけ実行されるクラスメソッド
        print("setUpClass実行")

    def test_example1(self):
        print("test_example1実行")

    def test_example2(self):
        print("test_example2実行")


unittest.main(argv=["first-arg-is-ignored"], exit=False)

FF..
FAIL: test_get_the_sum_of_two_integers (__main__.AddTest.test_get_the_sum_of_two_integers) [1 + 3 = 4] (idx=1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_21187/1848870928.py", line 12, in test_get_the_sum_of_two_integers
    self.assertEqual(add(a, b), expected)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
AssertionError: 3 != 4

FAIL: test_get_the_sum_of_two_integers (__main__.AddTest.test_get_the_sum_of_two_integers) [3 + 3 = 6] (idx=2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_21187/1848870928.py", line 12, in test_get_the_sum_of_two_integers
    self.assertEqual(add(a, b), expected)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
AssertionError: 7 != 6

----------------------------------------------------------------------
Ran 3 

setUpClass実行
setUp実行
test_example1実行
setUp実行
test_example2実行


In [39]:
class TearDownAndTearDownClassTest(unittest.TestCase):
    def tearDown(self):
        # テスト完了毎に実行されるメソッド
        print("tearDown実行")

    @classmethod
    def tearDownClass(cls):
        # テストの一番最後に1度のみ実行されるクラスメソッド
        print("tearDownClass実行")

    def test_example1(self):
        print("test_example1実行")

    def test_example2(self):
        print("test_example2実行")


unittest.main(argv=["first-arg-is-ignored"], exit=False)

FF....
FAIL: test_get_the_sum_of_two_integers (__main__.AddTest.test_get_the_sum_of_two_integers) [1 + 3 = 4] (idx=1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_21187/1848870928.py", line 12, in test_get_the_sum_of_two_integers
    self.assertEqual(add(a, b), expected)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
AssertionError: 3 != 4

FAIL: test_get_the_sum_of_two_integers (__main__.AddTest.test_get_the_sum_of_two_integers) [3 + 3 = 6] (idx=2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_21187/1848870928.py", line 12, in test_get_the_sum_of_two_integers
    self.assertEqual(add(a, b), expected)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
AssertionError: 7 != 6

----------------------------------------------------------------------
Ran 

setUpClass実行
setUp実行
test_example1実行
setUp実行
test_example2実行
test_example1実行
tearDown実行
test_example2実行
tearDown実行
tearDownClass実行


#### モックを利用してユニットテストを行う - unittest.mock


In [1]:
class ShoppingSiteAPI:
    """架空のショッピングサイトのAPIを呼ぶクラス"""

    def search_items(self, name: str) -> list[str]:
        """
        該当する名前の商品を検索する
        実際にはAPIの結果を返す必要があるが、架空のため固定値
        """
        return ["商品1", "商品2", "商品3"]

    def purchase(self, item_id: str):
        """
        商品を購入する
        実際には外部APIを呼ぶが、架空のため何も返さない
        """
        pass

In [2]:
# 単体テストを行いたい処理
def my_processing():
    api = ShoppingSiteAPI()
    return ",".join(api.search_items("商品")) + "が見つかりました"


print(my_processing)

<function my_processing at 0x10583d260>


- my_processing で使用される API をモックオブジェクトとして置き換えテストする


In [42]:
from unittest.mock import MagicMock

api = ShoppingSiteAPI()
api.search_items = MagicMock()

api.search_items

<MagicMock id='4600801536'>

In [43]:
# 関数の戻り値を設定
api.search_items.return_value = ["モック商品1", "モック商品2", "モック商品3"]

api.search_items("商品")

['モック商品1', 'モック商品2', 'モック商品3']

In [46]:
# 例外の設定
api.search_items.side_effect = Exception("例外を設定する")

api.search_items("商品")

Exception: 例外を設定する

- Mock と MagicMock の違いについて
  - MagicMock は Mock クラスのサブクラスとして定義されており、Mock クラスの持つ機能に加えて、Python のもつ特殊メソッドをサポートしている
  - 特に理由がない場合は MagicMock を利用する


In [48]:
from unittest.mock import Mock, MagicMock

magic_mock = MagicMock()
int(magic_mock)

1

In [49]:
mock = Mock()
int(mock)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Mock'

In [51]:
# intを利用したい場合は、特殊メソッド__int__を定義する必要がある
mock.__int__ = Mock(return_value=1)
int(mock)

1

#### クラスやメソッドをモックで置き換える - patch()


In [7]:
import unittest
from unittest.mock import patch


class ExampleTest(unittest.TestCase):
    # トップレベルのコードと同じ場所にモック対象がある場合は、明示的にそれを指定しないと参照してくれない
    @patch("__main__.ShoppingSiteAPI")
    def test_my_processing(self, APIMock):
        api = APIMock()
        api.search_items.return_value = ["モック商品1", "モック商品2", "モック商品3"]

        self.assertEqual(
            my_processing(), "モック商品1,モック商品2,モック商品3が見つかりました"
        )


unittest.main(argv=["first-arg-is-ignored"], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [8]:
class ExampleTest2(unittest.TestCase):
    def test_my_processing(self):
        with patch("__main__.ShoppingSiteAPI") as APIMock:
            api = APIMock()
            api.search_items.return_value = [
                "モック商品1",
                "モック商品2",
                "モック商品3",
            ]

            # コンテキストマネージャーを使用したアサート
            self.assertEqual(
                my_processing(), "モック商品1,モック商品2,モック商品3が見つかりました"
            )

        # パッチ適用されていない
        self.assertEqual(my_processing(), "商品1,商品2,商品3が見つかりました")


unittest.main(argv=["first-arg-is-ignored"], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


#### モックオブジェクトが呼び出されたか確認する


In [5]:
import unittest
from unittest.mock import patch


class ExampleTest3(unittest.TestCase):
    @patch("__main__.ShoppingSiteAPI.search_items")
    @patch("__main__.ShoppingSiteAPI.purchase")
    def test_example(self, purchace_mock, search_items_mock):
        search_items_mock.return_value = [
            "モック商品1",
            "モック商品2",
            "モック商品3",
        ]

        actual = my_processing()
        expected = "モック商品1,モック商品2,モック商品3が見つかりました"
        self.assertEqual(actual, expected)
        search_items_mock.assert_called()
        purchace_mock.assert_called()


unittest.main(argv=["first-arg-is-ignored"], exit=False)

F
FAIL: test_example (__main__.ExampleTest3.test_example)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/akagikouzanh/.pyenv/versions/3.13.0/lib/python3.13/unittest/mock.py", line 1423, in patched
    return func(*newargs, **newkeywargs)
  File "/var/folders/8f/5bctm_kd3qg_4dv_z9xc0lch0000gn/T/ipykernel_81075/2779125567.py", line 19, in test_example
    purchace_mock.assert_called()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/akagikouzanh/.pyenv/versions/3.13.0/lib/python3.13/unittest/mock.py", line 946, in assert_called
    raise AssertionError(msg)
AssertionError: Expected 'purchase' to have been called.

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (failures=1)


## Tips


In [10]:
%pip install freezegun


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# 日時取得するパターンのコードのテスト

from datetime import datetime
import unittest

import freezegun


class ExampleTest4(unittest.TestCase):
    @freezegun.freeze_time("2021-01-01 00:00:00")
    def test_example(self):
        self.assertEqual(datetime.utcnow(), datetime(2021, 1, 1, 0, 0, 0))

    def test_example2(self):
        with freezegun.freeze_time("2021-01-01 00:00:00"):
            self.assertEqual(datetime.utcnow(), datetime(2021, 1, 1, 0, 0, 0))


unittest.main(argv=["first-arg-is-ignored"], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.044s

OK
